<center><img src="../image/CLBLOGO.jpg" alt="创乐博" style="width: 300px;"/></center>

# 11.OpenCV 摄像头云台人脸追踪

@－－－－湖南创乐博智能科技有限公司－－－－<br>
@  文件名：11.OpenCV摄像头云台人脸追踪.ipynb <br>
@  版本：V2.0 <br>
@  author: zhulin<br>
@  说明：OpenCV摄像头云台人脸追踪<br>

## 为了可以打开摄像头，这里关闭摄像头所有进程

In [1]:
# 载入必要的库
import cv2
import numpy as np
from LOBOROBOT import LOBOROBOT # 载入机器人库
import time

In [2]:
clbrobot = LOBOROBOT()  # 实例化机器人对象
# Configure min and max servo pulse lengths
servo_min = 150  # Min pulse length out of 4096
servo_max = 600  # Max pulse length out of 4096

In [3]:
# 舵机调零
pan =  90
tilt = -10
# 频率设置为50hz，适用于舵机系统。
clbrobot.set_servo_angle(10,pan)  # 底座舵机 90 
clbrobot.set_servo_angle(9,tilt)  # 顶部舵机 30
time.sleep(0.5)

In [4]:
# 线程函数操作库
import threading # 线程
import ctypes
import inspect

# 线程结束代码
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [5]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

frame_img = widgets.Image(format='jpeg', width=320, height=240)
display(frame_img)

Image(value=b'', format='jpeg', height='240', width='320')

In [6]:
dispW=320
dispH=240

cam = cv2.VideoCapture(0,cv2.CAP_V4L2)
cam.set(3,320)
cam.set(4,240)
cam.isOpened()

width=cam.get(cv2.CAP_PROP_FRAME_WIDTH)
height=cam.get(cv2.CAP_PROP_FRAME_HEIGHT)
print('width:',width,'height:',height)

width: 320.0 height: 240.0


In [7]:
# 载入人脸和眼睛的HAAR 模型
face_cascade = cv2.CascadeClassifier('./image/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./image/haarcascade_eye.xml')

In [8]:
def Video_display():
    global pan
    global tilt
    while True: 
        ret,frame = cam.read()
        frame=cv2.flip(frame, 0)
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,1.3,5)

        for(x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
            Xcent = x + w/2
            Ycent = y + h/2
            errorPan  = Xcent - dispW/2 
            errorTilt = Ycent - dispH/2
            if abs(errorPan)>15:
                pan=pan-errorPan/50
            if abs(errorTilt)>15:
                tilt=tilt-errorTilt/50
            if pan > 180:
                pan = 180
                print("Pan out of Range")
            if pan < 0:
                pan = 0
                print("pan Out of Range")
            if tilt > 180:
                tilt = 180
                print("Pan out of Range")
            if tilt < 0:
                tilt = 0
                print("pan Out of Range")

            clbrobot.set_servo_angle(10,180-pan)
            clbrobot.set_servo_angle(9,-tilt)

            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]        
            eyes = eye_cascade.detectMultiScale(roi_gray)
            for (ex,ey,ew,eh) in eyes:
                cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)          
        frame_img.value = bgr8_to_jpeg(frame)
    cap.release()

In [9]:
# 开始线程
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

pan Out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Range
Pan out of Ran

In [14]:
# 结束线程
stop_thread(t)